In [21]:
import libsbml
import re
import pandas as pd
import numpy as np
from tools import *

In [22]:
model_name = "Zerrouk2024_M2"
sbml_file = f"Sources/{model_name}.xml"
save_file = f"Models/{model_name}_cleaned.sbml"

knowledge_source = 'pubmed'
qualifier = 'bqbiol:isDescribedBy'

## Add metaid to all components

In [23]:
reader = libsbml.SBMLReader()
document = reader.readSBML(sbml_file)
model = document.getModel().getPlugin('qual')

# Ensure each component in the SBML model has a metaid.
def generateMetaId(index):
    return f"metaid_{index:07d}"
    
index = 1
# Process the model
document.getModel().setMetaId(generateMetaId(index))

# Process species
for species in model.getListOfQualitativeSpecies():
    index += 1
    species.setMetaId(generateMetaId(index))

# Process transitions
for transition in model.getListOfTransitions():
    index += 1
    transition.setMetaId(generateMetaId(index))
    
    # set metaid for inputs of transition
    for input in transition.getListOfInputs():
        index += 1
        input.setMetaId(generateMetaId(index))
    
    # set metaid for outputs of transition
    for output in transition.getListOfOutputs():
        index += 1
        output.setMetaId(generateMetaId(index))

In [19]:
# libsbml.writeSBMLToFile(document, save_file)

1

## Only keep the pubmed annotation

Remove the "urn:casq" annotations.

In [24]:
# Create a dictionary for pubmed annotations for each species and transitions
pubmed_dict = {}
for species in model.getListOfQualitativeSpecies():
    annotations = species.getAnnotationString()
    if annotations:
        # Find all pubmed references in the format "urn:miriam:pubmed:NUMBERS"
        pubmed_refs = re.findall(r'urn:miriam:pubmed:(\d+)', annotations)
        # remove duplicates
        pubmed_refs = list(set(pubmed_refs))
        pubmed_dict[species.getMetaId()] = pubmed_refs

for transition in model.getListOfTransitions():
    annotations = transition.getAnnotationString()
    if annotations:
        # Find all pubmed references in the format "urn:miriam:pubmed:NUMBERS"
        pubmed_refs = re.findall(r'urn:miriam:pubmed:(\d+)', annotations)
        # remove duplicates
        pubmed_refs = list(set(pubmed_refs))
        pubmed_dict[transition.getMetaId()] = pubmed_refs

pubmed_dict

{'metaid_0000002': [],
 'metaid_0000003': ['8630106', '9824778'],
 'metaid_0000004': [],
 'metaid_0000005': ['22660635',
  '17009244',
  '16078317',
  '12847684',
  '11160335'],
 'metaid_0000006': [],
 'metaid_0000007': ['16973359', '8630106'],
 'metaid_0000008': ['8630106', '9824778', '8849348'],
 'metaid_0000009': [],
 'metaid_0000010': [],
 'metaid_0000011': ['12010572'],
 'metaid_0000012': ['27616146'],
 'metaid_0000013': ['28396660', '29495570', '12746896', '779025'],
 'metaid_0000014': ['29220376'],
 'metaid_0000015': ['31545398'],
 'metaid_0000016': ['25319955'],
 'metaid_0000017': ['28192374'],
 'metaid_0000018': [],
 'metaid_0000019': ['3240729', '25319955'],
 'metaid_0000020': [],
 'metaid_0000021': ['28192374', '31693422', '29749546'],
 'metaid_0000022': ['19480842', '23343469', '29471421'],
 'metaid_0000023': ['20415779', '30942097'],
 'metaid_0000024': [],
 'metaid_0000025': ['19126384', '29327326'],
 'metaid_0000026': [],
 'metaid_0000027': ['28228280', '30892966', '25975

In [25]:
# Set the annotations for each species and transition
for species in model.getListOfQualitativeSpecies():
    meta_id = species.getMetaId()
    if meta_id in pubmed_dict:
        if pubmed_dict[meta_id]:
            species.setAnnotation(createAnnotationString(qualifier, knowledge_source, RDF_TAG, pubmed_dict[meta_id], meta_id))
        else:
            species.unsetAnnotation()
    else:
        species.unsetAnnotation()

for transition in model.getListOfTransitions():
    meta_id = transition.getMetaId()
    if meta_id in pubmed_dict:
        if pubmed_dict[meta_id]:
            transition.setAnnotation(createAnnotationString(qualifier, knowledge_source, RDF_TAG, pubmed_dict[meta_id], meta_id))
        else:
            transition.unsetAnnotation()
    else:
        transition.unsetAnnotation()

libsbml.writeSBMLToFile(document, save_file)

1